# Importing Google Drive

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# Training CIFAR-10 dataset 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [3]:
!pip install torchinfo 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from torchinfo import summary
batch_size = 64
model = ResNet18()
summary(model, input_size=(batch_size, 3, 32, 32))

/usr/local/lib/python3.9/dist-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/usr/local/lib/python3.9/dist-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [64, 10]                  --
├─Conv2d: 1-1                            [64, 64, 32, 32]          1,728
├─BatchNorm2d: 1-2                       [64, 64, 32, 32]          128
├─Sequential: 1-3                        [64, 64, 32, 32]          --
│    └─BasicBlock: 2-1                   [64, 64, 32, 32]          --
│    │    └─Conv2d: 3-1                  [64, 64, 32, 32]          36,864
│    │    └─BatchNorm2d: 3-2             [64, 64, 32, 32]          128
│    │    └─Conv2d: 3-3                  [64, 64, 32, 32]          36,864
│    │    └─BatchNorm2d: 3-4             [64, 64, 32, 32]          128
│    │    └─Sequential: 3-5              [64, 64, 32, 32]          --
│    └─BasicBlock: 2-2                   [64, 64, 32, 32]          --
│    │    └─Conv2d: 3-6                  [64, 64, 32, 32]          36,864
│    │    └─BatchNorm2d: 3-7             [64, 64, 32, 32]          

In [9]:
# Define variables separately
batch_size = 128
test_batch_size = 128
epochs = 75
weight_decay = 2e-4
lr = 0.1
momentum = 0.9
no_cuda = False
seed = 1
model_dir = './model-cifar10-ResNet18'

# Settings
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [6]:
# Setup data loader for CIFAR-10
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),])

transform_test = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

100%|██████████| 170498071/170498071 [00:05<00:00, 29103923.52it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified


In [10]:
# Train function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # Calculate loss
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    return loss.item()

# Evaluation functions for train and test data
def eval_model(model, device, data_loader, loader_type, epoch):
    assert loader_type in ['train', 'test'], "loader_type must be either 'train' or 'test'"
    model.eval()
    data_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss(reduction='sum')
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            data_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    data_loss /= len(data_loader.dataset)

    print('Epoch: {}, {}: Average loss: {:.4f}, Accuracy: {:.0f}%'.format(epoch, 
                                                                          loader_type.capitalize(), 
                                                                          data_loss,
                                                                          100. * correct / len(data_loader.dataset)))

    data_accuracy = correct / len(data_loader.dataset)
    return data_loss, data_accuracy

In [11]:
dataset_name = 'cifar10'
print(f"Training on {dataset_name.upper()} dataset")
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=False, **kwargs)

model = ResNet18().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
best_test_accuracy = 0

for epoch in range(1, epochs + 1):

    # Training and updating losses and accuracies
    batch_train_loss = train(model, device, train_loader, optimizer, epoch)

    print('================================================================')
    avg_train_loss, train_accuracy = eval_model(model, device, train_loader, 'train', epoch)
    print("epoch:", epoch)
    print("train_losses:",avg_train_loss)
    print("train_accuracy:",train_accuracy)
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Testing and updating losses and accuracies
    avg_test_loss, test_accuracy = eval_model(model, device, test_loader, 'test', epoch)
    test_losses.append(avg_test_loss)
    test_accuracies.append(test_accuracy)

    # Save the model if the test accuracy is the best seen so far
    if test_accuracy > best_test_accuracy:
        best_test_accuracy = test_accuracy
        best_epoch = epoch
        torch.save(model.state_dict(), os.path.join(model_dir, f'model-ResNet18-{dataset_name}-best_epoch.pt'))
        torch.save(optimizer.state_dict(), os.path.join(model_dir, f'opt-ResNet18-{dataset_name}-checkpoint_best_epoch.tar'))

torch.save(model.state_dict(), 'checkpoint_final.pth')

Training on CIFAR10 dataset
Epoch: 1, Train: Average loss: 1.9057, Accuracy: 34%
epoch: 1
train_losses: 1.9057468228149415
train_accuracy: 0.34294
Epoch: 1, Test: Average loss: 1.9043, Accuracy: 36%
Epoch: 2, Train: Average loss: 1.3272, Accuracy: 52%
epoch: 2
train_losses: 1.3271535632324218
train_accuracy: 0.52484
Epoch: 2, Test: Average loss: 1.3274, Accuracy: 53%
Epoch: 3, Train: Average loss: 1.8285, Accuracy: 43%
epoch: 3
train_losses: 1.8284659283447267
train_accuracy: 0.4319
Epoch: 3, Test: Average loss: 1.9404, Accuracy: 42%
Epoch: 4, Train: Average loss: 1.0801, Accuracy: 62%
epoch: 4
train_losses: 1.0800559623718262
train_accuracy: 0.61664
Epoch: 4, Test: Average loss: 1.1191, Accuracy: 61%
Epoch: 5, Train: Average loss: 0.8221, Accuracy: 71%
epoch: 5
train_losses: 0.8221171780395508
train_accuracy: 0.70746
Epoch: 5, Test: Average loss: 0.8419, Accuracy: 70%
Epoch: 6, Train: Average loss: 0.7938, Accuracy: 72%
epoch: 6
train_losses: 0.7938214688873291
train_accuracy: 0.7236


In [12]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(epochs)), y=train_losses, name="Train Loss"))

fig.add_trace(go.Scatter(x=list(range(epochs)), y=test_losses, name="Test Loss"))
fig.update_layout(
    title="Model Loss",
    xaxis_title="Epoch",
    yaxis_title="Loss",
)

fig.show()

In [13]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(epochs)), y=train_accuracies, name="Train Accuracy"))

fig.add_trace(go.Scatter(x=list(range(epochs)), y=test_accuracies, name="Test Accuracy"))
fig.update_layout(
    title="Model Accuracy",
    xaxis_title="Epoch",
    yaxis_title="Accuracy",
)

fig.show()

In [14]:
!pip install git+https://github.com/fra31/auto-attack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-98t_hvon
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack /tmp/pip-req-build-98t_hvon
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=36249 sha256=5ac8dad262e1a811284002b0b21d46be706b420375875a41ac7e8d32fec7c3b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-e8ay87x5/wheels/e5/00/6a/fb12d1eaa81d79f8c0585bdddc361ca48c9633e9549db68aef
Successfully built autoattack


# AutoAttack - Linf

In [18]:
import os
import argparse
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms

from autoattack import AutoAttack


def load_model(model_path):
    model = ResNet18()
    ckpt = torch.load(model_path)
    model.load_state_dict(ckpt)
    model.cuda()
    model.eval()
    return model


def load_data(data_dir):
    transform_chain = transforms.Compose([transforms.ToTensor()])
    item = datasets.CIFAR10(root=data_dir, train=False, transform=transform_chain, download=True)
    test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)
    return test_loader


def create_save_dir(save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


def get_attack(model, norm, epsilon, log_path, version):
    adversary = AutoAttack(model, norm=norm, eps=epsilon,
                           log_path=log_path, version=version)
    return adversary

data_dir = './data'
norm = 'Linf'
epsilon = 8. / 255.
model_path = '/content/model-cifar10-ResNet18/model-ResNet18-cifar10-best_epoch.pt'    
n_ex = 1000
individual = False
save_dir = f'/content/aa-cifar-ResNet18-Linf/AA_results_cifar10_{norm}'
batch_size = 500

version = 'standard'
state_path = None

# Settings
parent_dir = os.path.dirname(save_dir)
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

log_path = os.path.join(save_dir, 'log_file.txt')

model = load_model(model_path)
test_loader = load_data(data_dir)
create_save_dir(save_dir)
adversary = get_attack(model, norm, epsilon, log_path, version)

l = [x for (x, y) in test_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in test_loader]
y_test = torch.cat(l, 0)

with torch.no_grad():
    if not individual:
        adv_complete = adversary.run_standard_evaluation(x_test[:n_ex], y_test[:n_ex],
                                                          bs=batch_size, state_path=state_path)

        torch.save({'adv_complete': adv_complete},
                    f'{save_dir}/CIFAR10_Linf_aa_{version}_1_{adv_complete.shape[0]}_eps_{epsilon:.5f}.pth')

    else:
        adv_complete = adversary.run_standard_evaluation_individual(x_test[:n_ex],
                                                                    y_test[:n_ex], bs=batch_size)

        torch.save(adv_complete, f'{save_dir}/CIFAR10_Linf_aa_{version}_individual_1_{n_ex}_eps_{epsilon:.5f}.pth')


Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 90.30%
apgd-ce - 1/2 - 500 out of 500 successfully perturbed
apgd-ce - 2/2 - 403 out of 403 successfully perturbed
robust accuracy after APGD-CE: 0.00% (total time 54.8 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 0.00%


# AutoAttack - L2

In [19]:
import os
import argparse
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms

from autoattack import AutoAttack


def load_model(model_path):
    model = ResNet18()
    ckpt = torch.load(model_path)
    model.load_state_dict(ckpt)
    model.cuda()
    model.eval()
    return model


def load_data(data_dir):
    transform_chain = transforms.Compose([transforms.ToTensor()])
    item = datasets.CIFAR10(root=data_dir, train=False, transform=transform_chain, download=True)
    test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)
    return test_loader


def create_save_dir(save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


def get_attack(model, norm, epsilon, log_path, version):
    adversary = AutoAttack(model, norm=norm, eps=epsilon,
                           log_path=log_path, version=version)
    return adversary

data_dir = './data'
norm = 'L2'
epsilon = 8. / 255.
model_path = '/content/model-cifar10-ResNet18/model-ResNet18-cifar10-best_epoch.pt'    
n_ex = 1000
individual = False
save_dir = f'/content/aa-cifar-ResNet18-Linf/AA_results_cifar10_{norm}'
batch_size = 500

version = 'standard'
state_path = None

# Settings
parent_dir = os.path.dirname(save_dir)
if not os.path.exists(parent_dir):
    os.makedirs(parent_dir)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

log_path = os.path.join(save_dir, 'log_file.txt')

model = load_model(model_path)
test_loader = load_data(data_dir)
create_save_dir(save_dir)
adversary = get_attack(model, norm, epsilon, log_path, version)

l = [x for (x, y) in test_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in test_loader]
y_test = torch.cat(l, 0)

with torch.no_grad():
    if not individual:
        adv_complete = adversary.run_standard_evaluation(x_test[:n_ex], y_test[:n_ex],
                                                          bs=batch_size, state_path=state_path)

        torch.save({'adv_complete': adv_complete},
                    f'{save_dir}/CIFAR10_L2_aa_{version}_1_{adv_complete.shape[0]}_eps_{epsilon:.5f}.pth')

    else:
        adv_complete = adversary.run_standard_evaluation_individual(x_test[:n_ex],
                                                                    y_test[:n_ex], bs=batch_size)

        torch.save(adv_complete, f'{save_dir}/CIFAR10_L2_aa_{version}_individual_1_{n_ex}_eps_{epsilon:.5f}.pth')


Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 90.30%
apgd-ce - 1/2 - 44 out of 500 successfully perturbed
apgd-ce - 2/2 - 42 out of 403 successfully perturbed
robust accuracy after APGD-CE: 81.70% (total time 53.9 s)
apgd-t - 1/2 - 3 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 317 successfully perturbed
robust accuracy after APGD-T: 81.40% (total time 494.6 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 314 successfully perturbed
robust accuracy after FAB-T: 81.40% (total time 1287.9 s)
square - 1/2 - 0 out of 500 successfully perturbed
square - 2/2 - 0 out of 314 successfully perturbed
robust accuracy after SQUARE: 81.40% (total time 2327.9 s)
max L2 perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 81.40%
